# Fine-Tune a Generative AI Model for Dialogue Summarization

The dialogue summary use case notebook, focuses on in-context learning while this notebook looks at modifying the model weights, specific for the summarization task in this notebook and to the loaded dataset.

This notebook utilizes the [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) model, known for its high-quality instruction-tuned capabilities and ability to summarize text without additional modifications. To enhance the model's performance, a full fine-tuning approach is implemented, and the results are evaluated using ROUGE metrics. Subsequently, the notebook investigates Parameter-Efficient Fine-Tuning (PEFT) techniques to fine-tune the model with a reduced number of parameters. The performance of the PEFT model is then evaluated and compared to the fully fine-tuned model, demonstrating that PEFT's advantages in terms of efficiency often outweigh the potential for slightly lower performance metrics. 

The goal of this notebook is to get to the point where one simple call into our model can give us a decent summary without having to pass in the one shot and few shot examples.

## 1 - Install and Import Required Dependencies, Dataset and LLM

### 1.1 - Install and Import Required Dependencies
Leverage the following libraries:
- PyTorch framework
- Torchdata for PyTorch data loading
- Evaluate library to calculate rouge score
- Parameter efficient fine-tuning with loralib and peft


In [3]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==2.5.1 \
    torchdata==0.10.1 --quiet

%pip install \
    transformers==4.47.1 \
    datasets==3.2.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Import the necessary libraries.

Note: AutoModelForSeq2SeqLM enables access to FLAN-T5. AutoModelForSeq2SeqLM can be used to load any seq2seq (or encoder-decoder) model that has a language modeling (LM) head on top. These include BART, PEGASUS, T5, etc. Full list of supported models in the docs available here: [Auto Classes](https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoModelForSeq2SeqLM).

TrainingArguments and Trainer are classes that help simplify the LLM training / fine-tuning.

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

c:\Users\rg255041\AppData\Local\anaconda3\envs\PEFT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 1.2 - Load Dataset and LLM

This notebook experiments with the [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face dataset. It contains 10,000+ dialogues with the corresponding manually labeled summaries and topics. 

In [3]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

Load the pre-trained [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5) and its tokenizer directly from HuggingFace. Notice that this notebook will be using the [small version](https://huggingface.co/google/flan-t5-base) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

```original_model``` will be used as a baseline for comparison for the other fine-tuning strategies. 

In [4]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

The following convenience function pulls out the number of model parameters and find out how many of them are trainable.

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params:,}\nall model parameters: {all_model_params:,}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247,577,856
all model parameters: 247,577,856
percentage of trainable model parameters: 100.00%


### 1.3 - Test the Model with Zero Shot Inferencing

Test the model with the zero shot inferencing. The model struggles to summarize the dialogue compared to the baseline summary, but it does pull out some important information from the text which indicates the model can be fine-tuned to the task at hand.

In [6]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------

## 2 - Perform Full Fine-Tuning

### 2.1 - Preprocess the Dialog-Summary Dataset

The following convenience function converts the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. It will tokenize and wrap the dataset into a prompt. It will prepend an instruction to the start of the dialog with `Summarize the following conversation` and to the start of the summary with `Summary` as follows:

Training prompt (dialogue):
```
Summarize the following conversation.

    Chris: This is his part of the conversation.
    Antje: This is her part of the conversation.
    
Summary: 
```

Training response (summary):
```
Both Chris and Antje participated in the conversation.
```

Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

This function will be used to iterate through all elements in the dataset to convert them into prompts with instruction (fine-tuning with instruction prompts).

In [7]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map: 100%|██████████| 500/500 [00:00<00:00, 2403.76 examples/s]


To save some time in the lab, subsample the dataset:

In [8]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter: 100%|██████████| 500/500 [00:00<00:00, 2383.86 examples/s]


Check the shapes of all three parts of the dataset:

In [9]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


The output dataset is ready for fine-tuning.

### 2.2 - Fine-Tune the Model with the Preprocessed Dataset

This section utilizes the built-in Hugging Face `Trainer` class (see the documentation [here](https://huggingface.co/docs/transformers/main_classes/trainer)). Pass the preprocessed dataset with reference to the original model.

max_steps and num_train_epochs are kept at a minimum to reduce resource consumption. 

In [10]:
output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

Start training process.

In [11]:
trainer.train()

100%|██████████| 1/1 [00:56<00:00, 56.85s/it]

{'loss': 47.5, 'grad_norm': 414.0, 'learning_rate': 0.0, 'epoch': 0.06}


100%|██████████| 1/1 [00:58<00:00, 58.03s/it]

{'train_runtime': 58.0398, 'train_samples_per_second': 0.138, 'train_steps_per_second': 0.017, 'train_loss': 47.5, 'epoch': 0.06}


TrainOutput(global_step=1, training_loss=47.5, metrics={'train_runtime': 58.0398, 'train_samples_per_second': 0.138, 'train_steps_per_second': 0.017, 'total_flos': 5478058819584.0, 'train_loss': 47.5, 'epoch': 0.0625})

Training a fully fine-tuned version of the model would take a few hours on a GPU. To save time, download a checkpoint of the fully fine-tuned model to use in the rest of this notebook. This fully fine-tuned model will also be referred to as the **instruct model** in this lab.

In [12]:
instruct_model_name="truocpham/flan-dialogue-summary-checkpoint"

instruct_model = AutoModelForSeq2SeqLM.from_pretrained( instruct_model_name, torch_dtype=torch.bfloat16)

c:\Users\rg255041\AppData\Local\anaconda3\envs\PEFT\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rg255041\.cache\huggingface\hub\models--truocpham--flan-dialogue-summary-checkpoint. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


### 2.3 - Evaluate the Model Qualitatively (Human Evaluation)

As with many GenAI applications, a qualitative approach where you ask yourself the question "Is my model behaving the way it is supposed to?" is usually a good starting point. The example below shows how the fine-tuned model is able to create a reasonable summary of the dialogue compared to the original inability to understand what is being asked of the model.

In [26]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL (ZERO SHOT):\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL (ZERO SHOT):
# Person1#: I'm not sure what I would need. #Person2#: I could make my software better. #Person1#: I'd like to make my own flyers and banners for advertising. #Person1#: I'd like a faster processor, more memory and a faster modem. #Person1#: I'd like to make up your own flyers and banners for advertising. #Person2#: I'd like to make up your own flyers and banners for advertising. #Person1#: I'd like to make up your own flyers and banners. #Person2#: I'd like to make up your own flyers and banners for advertising. #Person1#: I'd like to make up your own flyers and banners for advertising. #Person2#: I'd like to make up
----------------------------------

### 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness accomplished by fine-tuning.

In [14]:
rouge = evaluate.load('rouge')

Generate the outputs for the sample of the test dataset (only 10 dialogues and summaries to save time), and save the results.

In [27]:
dialogues = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_zero_shot_summaries', 'instruct_model_summaries'])
df

,human_baseline_summaries,original_model_zero_shot_summaries,instruct_model_summaries
0,Ms. Dawson helps #Person1# to write a memo to ...,The new policy is effective immediately.,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,Employees are required to take a dictation on ...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,The memo is to be distributed to all employees...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,The car is a nuisance.,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,The car is a problem in the city.,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,#Person1#: I'm not sure I can drive to work. I...,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,# Person1#: I still can't believe what happene...,Masha and Hero are getting divorced. Kate can'...
7,#Person1# tells Kate that Masha and Hero are g...,#Person1#: Masha and Hero are getting divorced...,Masha and Hero are getting divorced. Kate can'...
8,#Person1# and Kate talk about the divorce betw...,#Person1: Masha and Hero are getting divorced....,Masha and Hero are getting divorced. Kate can'...
9,#Person1# and Brian are at the birthday party ...,"#Person1: Thank you for your birthday wishes, ...",Brian's birthday is coming. #Person1# invites ...


Evaluate the models computing ROUGE metrics. Notice the improvement in the results. With some fine tuning using the dataset and a specific prompt, the rouge metric improves. 

In [16]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2238339018123694), 'rouge2': np.float64(0.060397216201981324), 'rougeL': np.float64(0.18625460316351472), 'rougeLsum': np.float64(0.18542336818193922)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.41026607717457186), 'rouge2': np.float64(0.17840645241958838), 'rougeL': np.float64(0.2977022096267017), 'rougeLsum': np.float64(0.2987374187518165)}


The file `data/dialogue-summary-training-results.csv` contains a pre-populated list of all model results which you can use to evaluate on a larger section of data (output of the above code that was executed on the full dataset). With this, we are able to evaluate the model on the full dataset, without having to inference on each dialogue. 

In [24]:
results = pd.read_csv("../data/dialogue-summary-training-results.csv")

human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.2334158581572823), 'rouge2': np.float64(0.07603964187010573), 'rougeL': np.float64(0.20145520923859048), 'rougeLsum': np.float64(0.20145899339006135)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.42161291557556113), 'rouge2': np.float64(0.18035380596301792), 'rougeL': np.float64(0.3384439349963909), 'rougeLsum': np.float64(0.33835653595561666)}


The results show substantial improvement in all ROUGE metrics:

In [25]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE
rouge1: 18.82%
rouge2: 10.43%
rougeL: 13.70%
rougeLsum: 13.69%
